# 1. Packages <a class="anchor" id="id1"></a>

In [1]:
import numpy as np
import imageio as io
import pandas as pd
import matplotlib.pyplot as plt


#.py files
import BCDR_utils as utils
import lesion_patching as patches
import generate_data as generator

ModuleNotFoundError: No module named 'BCDR_utils'

# 2. CSV Data <a class="anchor" id="id2"></a>

### 3.1. BCDR Dataset 1<a class="anchor" id="id3.1"></a>

In [ ]:
bcdr1_raw = pd.read_csv('BCDR-D01_dataset/bcdr_d01_img.csv')
bcdr1_raw= patches.fix_view(bcdr1_raw,'image_type_name')
bcdr1_features_raw = pd.read_csv('BCDR-D01_dataset/bcdr_d01_features.csv')
print('Suspicious Dataset Nº 1: ',bcdr1_raw.shape)
bcdr1_features_raw.head(5)

### 3.3. BCDR Dataset 2  <a class="anchor" id="id3.2"></a>

In [ ]:
bcdr2_raw = pd.read_csv('BCDR-D02_dataset/bcdr_d02_img.csv')
bcdr2_raw= patches.fix_view(bcdr2_raw,'image_type_name')
bcdr2_features_raw = pd.read_csv('BCDR-D02_dataset/bcdr_d02_features.csv')
print('Suspicious Dataset Nº 2: ',bcdr2_raw.shape)
bcdr2_raw.head(5)

## Lesion Center and Area

In [ ]:
bcdr1 = bcdr1_raw[['patient_id','study_id','image_filename','image_type_name']]
bcdr1_features= bcdr1_features_raw[['patient_id','study_id','image_view','s_x_center_mass','s_y_center_mass','s_area']]

bcdr1_final = patches.merge_csv(bcdr1_features,bcdr1)

In [ ]:
bcdr2 = bcdr2_raw[['patient_id','study_id','image_filename','image_type_name']]
bcdr2_features= bcdr2_features_raw[['patient_id','study_id','image_view','s_x_center_mass','s_y_center_mass','s_area']]

bcdr2_final = patches.merge_csv(bcdr2_features,bcdr2)

## Tiff Paths

In [ ]:
broken_paths_1= []
for path in bcdr1_final['image_filename']:
    broken_paths_1.append('BCDR-D01_dataset/'+path)
suspicious_paths_1_tif = utils.fix_path(broken_paths_1)

suspicious_paths_1= generator.convert(suspicious_paths_1_tif)

In [ ]:
suspicious_broken_paths_2= []
for path in bcdr2_final['image_filename']:
    suspicious_broken_paths_2.append('BCDR-D02_dataset/'+path)
suspicious_paths_2_tif = utils.fix_path(suspicious_broken_paths_2)

suspicious_paths_2 = generator.convert(suspicious_paths_2_tif)

In [ ]:
#bcdr1_final.head(50)

## Lesion Patching

In [ ]:
path_to_patches_1,unsaved_patches_1 = patches.patching_lesions(bcdr1_final,suspicious_paths_1)

In [ ]:
path_to_patches_2,unsaved_patches_2 = patches.patching_lesions(bcdr2_final,suspicious_paths_2)

## Unsaved Patches?

## Amostra de Lesion Patching

In [ ]:
patient = 20

In [ ]:
a= np.asarray(io.imread(suspicious_paths_1[patient]))
x= a.shape[1]*bcdr1_features.loc[patient,'s_x_center_mass']
y= a.shape[0]*bcdr1_features.loc[patient,'s_y_center_mass']

In [ ]:
i=plt.imshow(a,cmap="gray")
plt.scatter(x,y,c='r')
plt.show()

In [ ]:
center_lesion = (int(round(x)),int(round(y)))
area_lesion = bcdr1_features.loc[patient,'s_area']
side_lesion = int(round(np.sqrt(area_lesion)/2)+25)

In [ ]:
patch = a[center_lesion[1]-side_lesion:center_lesion[1]+side_lesion,center_lesion[0]-side_lesion:center_lesion[0]+side_lesion]

In [ ]:
plt.imshow(patch)

## Generating Patches

## Building Normal vs Suspicious Dataframes

In [ ]:
bcdr_normal = pd.DataFrame({'paths':normal_paths,'labels':'Normal'})

In [ ]:
#suspicious_dfs = [pd.DataFrame({'paths':suspicious_paths_1,'labels':'Suspicious'}),pd.DataFrame({'paths':suspicious_paths_2,'labels':'Suspicious'})]

#bcdr_suspicious = pd.concat(suspicious_dfs,ignore_index=True)

In [ ]:
bcdr_suspicious = pd.DataFrame({'paths':suspicious_paths_1,'labels':'Suspicious'})

## Building Training vs Validation Dataframes

In [ ]:
validation_split=0.2
validation_images = round(0.2*(bcdr_normal.shape[0]+bcdr_suspicious.shape[0]))
print('Validation Images: ',round(validation_images))

### Starts IEETA

In [ ]:
validation_suspicious = bcdr_suspicious[:round((validation_images/2))]
validation_normal = bcdr_normal[:round((validation_images/2))]
training_suspicious = bcdr_suspicious[round(validation_images/2):]
training_normal = bcdr_normal[round(validation_images/2):]

### Ends IEETA

In [ ]:
validation_df = pd.concat([
    bcdr_suspicious[:round((validation_images/2))],
    bcdr_normal[:round((validation_images/2))]],
    ignore_index=True)

In [ ]:
training_df = pd.concat([
    bcdr_suspicious[round(validation_images/2):],
    bcdr_normal[round(validation_images/2):]],
    ignore_index=True)

## Batching Data

In [ ]:
target_size_1 = 2500
target_size_2 = 2000
batch_size = 16

In [ ]:
training_generator = generator.generate(training_df,target_size_1,target_size_2,batch_size,c_mode='grayscale')
validation_generator = generator.generate(validation_df,target_size_1,target_size_2,batch_size,c_mode='grayscale')

### Starts IEETA

In [ ]:
validation_suspicious_generator = generator.generate_and_save(validation_suspicious,target_size_1,target_size_2,batch_size,'grayscale',"D:/BCDR/Full Mamografies/validation_data/suspicious")
validation_normal_generator = generator.generate_and_save(validation_normal,target_size_1,target_size_2,batch_size,'grayscale',"D:/BCDR/Full Mamografies/validation_data/normal")
training_suspicious_generator = generator.generate_and_save(training_suspicious,target_size_1,target_size_2,batch_size,'grayscale',"D:/BCDR/Full Mamografies/training_data/suspicious")
training_normal_generator = generator.generate_and_save(training_normal,target_size_1,target_size_2,batch_size,'grayscale',"D:/BCDR/Full Mamografies/training_data/normal")

In [ ]:
for i in range(len(validation_suspicious_generator)):
    validation_suspicious_generator[i]

In [ ]:
for i in range(len(validation_normal_generator)):
    validation_normal_generator[i]

In [ ]:
for i in range(len(training_normal_generator)):
    print(i)
    training_normal_generator[i]

In [ ]:
for i in range(len(training_suspicious_generator)):
    print(i)
    training_suspicious_generator[i]

### Ends IEETA